# MeiliSearchVectorStore

```{=mdx}
:::tip Compatibility
Only available on Node.js.
:::
```

[MeiliSearch](https://www.meilisearch.com/) is a open-source flexible and powerful user-focused search engine that is lightning fast. It is designed for developers and provides an easy-to-use search engine. 

This is a quick start guide for using Meilisearch [vector stores](/docs/concepts/#vectorstores) with LangchainJS. For detailed documentation of all `MeiliSearchVectorStore` features and configurations head to the [API reference](https://api.js.langchain.com/classes/langchain_meilisearch.MeiliSearchVectorStore.html).

## Overview

### Integration details

| Class | Package | [PY support](https://python.langchain.com/docs/integrations/vectorstores/meilisearch/) |  Package latest |
| :--- | :--- | :---: | :---: |
| [`MeiliSearchVectorStore`](https://api.js.langchain.com/classes/langchain_meilisearch.MeiliSearchVectorStore.html) | [`@langchain/meilisearch`](https://npmjs.com/@langchain/meilisearch) | ✅ |  ![NPM - Version](https://img.shields.io/npm/v/@langchain/meilisearch?style=flat-square&label=%20&) |

## Setup

To use MeiliSearch vector store, you'll need to set up a Meilisearch instance and install the `@langchain/meilisearch` integration package.

You can run a Meilisearch instance with Docker on your computer by following the [Meilisearch setup instructions](https://www.meilisearch.com/docs/guides/misc/docker). Note the URL your container runs on. Or you can run a Meilisearch instance directly on your computer downloading the [Meilisearch binary](https://github.com/meilisearch/meilisearch/releases). from Github.

**Make a note of the URL your Meilisearch instance is running on and the Master key of your Meilisearch instance. You'll need these to connect to it from LangchainJS.**

This guide will also use [OpenAI embeddings](/docs/integrations/text_embedding/openai), which require you to install the `@langchain/openai` integration package. You can also use [other supported embeddings models](/docs/integrations/text_embedding) if you wish.

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/meilisearch @langchain/core @langchain/openai
</Npm2Yarn>
```

### Credentials

Once you've done the above setup, you can use the following code to create the Meilisearch instance:

```typescript
// default host: http://localhost:7700
const meilisearch_client = new MeiliSearch({
  host: 'MEILISEARCH_HOST',
  apiKey: 'MEILISEARCH_MASTER_KEY',
});
```

You will be passing this client to the `MeiliSearchVectorStore` constructor.

## Instantiation

This library provides 3 ways to instantiate a `MeiliSearchVectorStore`:

1. From Texts
2. From Documents
3. From an Existing MeiliSearch Index

## 

### From Texts

This is the simplest way to create a `MeiliSearchVectorStore`. You can pass an array of
text strings to the constructor, and the store will create vectors for each text.
Optionally, you can pass an array of Metadata objects to associate metadata with each text.

In [1]:
import { MeiliSearchVectorStore } from "@langchain/meilisearch";
import { OpenAIEmbeddings } from "@langchain/openai";

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
});

const meilisearch_client = new MeiliSearch({
  host: 'MEILISEARCH_HOST',
  apiKey: 'MEILISEARCH_MASTER_KEY',
});

const documentTexts = [
  "I feel great today!",
  "I feel sad today.",
  "I feel happy today.",
];

const vectorStore = await MeiliSearchVectorStore.fromTexts(documentTexts,{}, embeddings, {
  client: meilisearch_client,
  index_name: "langchainjs-meilisearch",
});

### From Documents

You can also create a `MeiliSearchVectorStore` from an array of langchain documents. Each document
should be an object with a `pageContent` key containing the text to create a vector for. Optionally,
you can pass the `metadata` key to associate metadata with each document.

In [ ]:
import { MeiliSearchVectorStore } from "@langchain/meilisearch";
import { OpenAIEmbeddings } from "@langchain/openai";

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
});

const meilisearch_client = new MeiliSearch({
  host: 'MEILISEARCH_HOST',
  apiKey: 'MEILISEARCH_MASTER_KEY',
});

const documents = [
    {
        "pageContent": "I feel great today!",
        "metadata": {}
    },
    {
        "pageContent": "I feel sad today.",
        "metadata": {}
    },
    {
        "pageContent": "I feel happy today.",
        "metadata": {}
    }
];

const vectorStore = await MeiliSearchVectorStore.fromDocuments(documents,embeddings, {
  client: meilisearch_client,
  index_name: "langchainjs-meilisearch",
});

### From an Existing MeiliSearch Index

You can also create a `MeiliSearchVectorStore` from an existing MeiliSearch index. This is useful
if you want to reuse an existing index for vector storage. You can pass the index name to the constructor.

In [ ]:
import { MeiliSearchVectorStore } from "@langchain/meilisearch";
import { OpenAIEmbeddings } from "@langchain/openai";

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
});

const meilisearch_client = new MeiliSearch({
  host: 'MEILISEARCH_HOST',
  apiKey: 'MEILISEARCH_MASTER_KEY',
});

const vectorStore = await MeiliSearchVectorStore.fromExistingIndex(embeddings, {
  client: meilisearch_client,
  index_name: "langchainjs-meilisearch",
});

## Manage vector store

### Add items to vector store

In [3]:
import type { Document } from "@langchain/core/documents";

const document1: Document = {
  pageContent: "The powerhouse of the cell is the mitochondria",
  metadata: { source: "https://example.com" }
};

const document2: Document = {
  pageContent: "Buildings are made out of brick",
  metadata: { source: "https://example.com" }
};

const document3: Document = {
  pageContent: "Mitochondria are made out of lipids",
  metadata: { source: "https://example.com" }
};

const document4: Document = {
  pageContent: "The 2024 Olympics are in Paris",
  metadata: { source: "https://example.com" }
}

const documents = [document1, document2, document3, document4];

await vectorStore.addDocuments(documents);

## Query vector store

Once your vector store has been created and the relevant documents have been added you will most likely wish to query it during the running of your chain or agent. 

### Query directly

Performing a simple similarity search can be done as follows:

In [7]:
const filter = {
  "must": [
      { "key": "metadata.source", "match": { "value": "https://example.com" } },
  ]
};

const similaritySearchResults = await vectorStore.similaritySearch("biology", 2, filter);

for (const doc of similaritySearchResults) {
  console.log(`* ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}

* The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* Mitochondria are made out of lipids [{"source":"https://example.com"}]


See [this page](https://www.meilisearch.com/docs/learn/filtering_and_sorting/filter_search_results) for more on Meilisearch filter syntax.

If you want to execute a similarity search and receive the corresponding scores you can run:

In [8]:
const similaritySearchWithScoreResults = await vectorStore.similaritySearchWithScore("biology", 2, filter)

for (const [doc, score] of similaritySearchWithScoreResults) {
  console.log(`* [SIM=${score.toFixed(3)}] ${doc.pageContent} [${JSON.stringify(doc.metadata)}]`);
}

* [SIM=0.165] The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* [SIM=0.148] Mitochondria are made out of lipids [{"source":"https://example.com"}]


### Query by turning into retriever

You can also transform the vector store into a [retriever](/docs/concepts/#retrievers) for easier usage in your chains. 

In [9]:
const retriever = vectorStore.asRetriever({
  // Optional filter
  filter: filter,
  k: 2,
});
await retriever.invoke("biology");

[
  Document {
    pageContent: 'The powerhouse of the cell is the mitochondria',
    metadata: { source: 'https://example.com' },
    id: undefined
  },
  Document {
    pageContent: 'Mitochondria are made out of lipids',
    metadata: { source: 'https://example.com' },
    id: undefined
  }
]


## Manage Indexes

You can also manage the indexes in your MeiliSearch instance using the this library.

```typescript

// Delete the index.
await vectorStore.deleteIndex();

// Delete all Documents in the index.
await vectorStore.deleteAllDocuments();

// Enable Vector Store functionality on the Instance.
await vectorStore.enableVectorStore();

// Check if the Instance is Healthy.
await vectorStore.isHealthy();

```

### Usage for retrieval-augmented generation

For guides on how to use this vector store for retrieval-augmented generation (RAG), see the following sections:

- [Tutorials: working with external knowledge](/docs/tutorials/#working-with-external-knowledge).
- [How-to: Question and answer with RAG](/docs/how_to/#qa-with-rag)
- [Retrieval conceptual docs](/docs/concepts#retrieval)

## API reference

For detailed documentation of all `MeiliSearchVectorStore` features and configurations head to the [API reference](https://api.js.langchain.com/classes/langchain_meilisearch.MeiliSearchVectorStore.html).